In [50]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import os
import datetime

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [51]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three


In [0]:
train = pd.read_pickle('data/train.p')
X_train, y_train = train['features'], train['labels']
test = pd.read_pickle('data/test_road_sign.p')
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1:
  y_train = to_categorical(y_train)

In [0]:
if y_test.ndim == 1:
  y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def get_cnn_v1(input_shape, num_classes):
  return Sequential([
          Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=input_shape),

          Flatten(),

          Dense(num_classes, activation='softmax'),
])
  
def get_cnn_v2(input_shape, num_classes):
  return Sequential([
          Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
          MaxPool2D(),
          Dropout(0.5),

          Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
          MaxPool2D(),
          Dropout(0.3),

          Flatten(),

          Dense(1024, activation='relu'),
          Dropout(0.5),

          Dense(num_classes, activation='softmax'),
])
  
def train_model(model, X_train, y_train, params_fit=None):
  if params_fit is None:
    params_fit = {}

  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose',1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
  )

  return model

In [57]:
model = get_cnn_v1(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

Epoch 1/5
272/272 [==============================] - 7s 24ms/step - loss: 31.0326 - accuracy: 0.7506 - val_loss: 0.2374 - val_accuracy: 0.9446
Epoch 2/5
272/272 [==============================] - 6s 23ms/step - loss: 0.2304 - accuracy: 0.9507 - val_loss: 0.1663 - val_accuracy: 0.9618
Epoch 3/5
272/272 [==============================] - 6s 23ms/step - loss: 0.1425 - accuracy: 0.9672 - val_loss: 0.0963 - val_accuracy: 0.9782
Epoch 4/5
272/272 [==============================] - 6s 23ms/step - loss: 0.1345 - accuracy: 0.9721 - val_loss: 0.1664 - val_accuracy: 0.9674
Epoch 5/5
272/272 [==============================] - 6s 23ms/step - loss: 0.1047 - accuracy: 0.9776 - val_loss: 0.0434 - val_accuracy: 0.9895


In [0]:
df = pd.read_csv('data/signnames.csv')
labels_dict = df.to_dict()['b']

In [59]:
model = get_cnn_v2(input_shape, num_classes)
params_fit = {
    'epochs':25
}
model_trained_2 = train_model(model, X_train, y_train,params_fit)

Epoch 1/25
272/272 [==============================] - 7s 25ms/step - loss: 8.6143 - accuracy: 0.0863 - val_loss: 3.0792 - val_accuracy: 0.1539
Epoch 2/25
272/272 [==============================] - 6s 23ms/step - loss: 2.8492 - accuracy: 0.2273 - val_loss: 2.3242 - val_accuracy: 0.3730
Epoch 3/25
272/272 [==============================] - 6s 23ms/step - loss: 1.7605 - accuracy: 0.4873 - val_loss: 1.0018 - val_accuracy: 0.7419
Epoch 4/25
272/272 [==============================] - 6s 23ms/step - loss: 1.1198 - accuracy: 0.6528 - val_loss: 0.5439 - val_accuracy: 0.8642
Epoch 5/25
272/272 [==============================] - 6s 23ms/step - loss: 0.8324 - accuracy: 0.7382 - val_loss: 0.3856 - val_accuracy: 0.9025
Epoch 6/25
272/272 [==============================] - 6s 23ms/step - loss: 0.6705 - accuracy: 0.7907 - val_loss: 0.2516 - val_accuracy: 0.9397
Epoch 7/25
272/272 [==============================] - 6s 23ms/step - loss: 0.5698 - accuracy: 0.8217 - val_loss: 0.1987 - val_accuracy: 0.9555

In [0]:
y_test_norm = np.argmax(y_test, axis=1)
y_pred_prob = model_trained.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)


In [62]:
predict(model_trained, X_test, y_test)

0.8394557823129252

In [63]:
predict(model_trained_2, X_test, y_test)

0.9433106575963719

In [0]:
def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [65]:
def get_cnn_v3(input_shape, num_classes):
  return Sequential([
          Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
          MaxPool2D(),
          Dropout(0.3),

          Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
          MaxPool2D(),
          Dropout(0.3),

          Flatten(),

          Dense(1024, activation='relu'),
          Dropout(0.3),

          Dense(num_classes, activation='softmax'),
])



train_and_predict(get_cnn_v3(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 7s 24ms/step - loss: 4.1449 - accuracy: 0.3654 - val_loss: 1.0333 - val_accuracy: 0.7106
Epoch 2/5
272/272 [==============================] - 6s 23ms/step - loss: 0.9559 - accuracy: 0.7132 - val_loss: 0.2855 - val_accuracy: 0.9459
Epoch 3/5
272/272 [==============================] - 6s 23ms/step - loss: 0.4904 - accuracy: 0.8548 - val_loss: 0.1238 - val_accuracy: 0.9755
Epoch 4/5
272/272 [==============================] - 6s 23ms/step - loss: 0.3368 - accuracy: 0.9025 - val_loss: 0.0728 - val_accuracy: 0.9835
Epoch 5/5
272/272 [==============================] - 6s 23ms/step - loss: 0.2510 - accuracy: 0.9273 - val_loss: 0.0579 - val_accuracy: 0.9900


0.9095238095238095

In [66]:
def get_cnn_v3(input_shape, num_classes):
  return Sequential([
          Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
          Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
          MaxPool2D(),
          Dropout(0.3),

          Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
          Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
          MaxPool2D(),
          Dropout(0.3),

          Flatten(),

          Dense(1024, activation='relu'),
          Dropout(0.3),

          Dense(num_classes, activation='softmax'),
])



train_and_predict(get_cnn_v3(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 10s 37ms/step - loss: 2.3600 - accuracy: 0.5463 - val_loss: 0.2680 - val_accuracy: 0.9445
Epoch 2/5
272/272 [==============================] - 10s 36ms/step - loss: 0.4061 - accuracy: 0.8848 - val_loss: 0.0889 - val_accuracy: 0.9817
Epoch 3/5
272/272 [==============================] - 10s 36ms/step - loss: 0.2301 - accuracy: 0.9336 - val_loss: 0.0691 - val_accuracy: 0.9897
Epoch 4/5
272/272 [==============================] - 10s 36ms/step - loss: 0.1598 - accuracy: 0.9543 - val_loss: 0.0247 - val_accuracy: 0.9935
Epoch 5/5
272/272 [==============================] - 10s 37ms/step - loss: 0.1237 - accuracy: 0.9642 - val_loss: 0.0170 - val_accuracy: 0.9957


0.9736961451247166

In [67]:
def get_cnn_v4(input_shape, num_classes):
  return Sequential([
          Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
          Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
          MaxPool2D(),
          Dropout(0.3),

          Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
          Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
          MaxPool2D(),
          Dropout(0.3),

          Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
          Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
          MaxPool2D(),
          Dropout(0.3),

          Flatten(),

          Dense(1024, activation='relu'),
          Dropout(0.3),

          Dense(num_classes, activation='softmax'),
])



train_and_predict(get_cnn_v4(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 12s 44ms/step - loss: 2.5446 - accuracy: 0.3037 - val_loss: 0.7912 - val_accuracy: 0.7642
Epoch 2/5
272/272 [==============================] - 12s 43ms/step - loss: 0.6473 - accuracy: 0.7969 - val_loss: 0.1018 - val_accuracy: 0.9760
Epoch 3/5
272/272 [==============================] - 12s 43ms/step - loss: 0.2664 - accuracy: 0.9197 - val_loss: 0.0500 - val_accuracy: 0.9874
Epoch 4/5
272/272 [==============================] - 12s 43ms/step - loss: 0.1593 - accuracy: 0.9523 - val_loss: 0.0273 - val_accuracy: 0.9922
Epoch 5/5
272/272 [==============================] - 12s 43ms/step - loss: 0.1287 - accuracy: 0.9627 - val_loss: 0.0207 - val_accuracy: 0.9941


0.9693877551020408

In [68]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
          Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
          Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
          MaxPool2D(),
          Dropout(0.3),

          Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
          Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
          MaxPool2D(),
          Dropout(0.3),
          
          Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
          Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
          MaxPool2D(),
          Dropout(0.3),

          Flatten(),

          Dense(1024, activation='relu'),
          Dropout(0.3),

          Dense(1024, activation='relu'),
          Dropout(0.3),

          Dense(num_classes, activation='softmax'),
])



train_and_predict(get_cnn_v5(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 13s 47ms/step - loss: 2.6796 - accuracy: 0.2655 - val_loss: 1.1367 - val_accuracy: 0.6096
Epoch 2/5
272/272 [==============================] - 12s 46ms/step - loss: 0.9199 - accuracy: 0.7036 - val_loss: 0.2600 - val_accuracy: 0.9170
Epoch 3/5
272/272 [==============================] - 12s 45ms/step - loss: 0.4520 - accuracy: 0.8564 - val_loss: 0.1299 - val_accuracy: 0.9547
Epoch 4/5
272/272 [==============================] - 12s 46ms/step - loss: 0.2863 - accuracy: 0.9116 - val_loss: 0.0646 - val_accuracy: 0.9808
Epoch 5/5
272/272 [==============================] - 12s 45ms/step - loss: 0.2264 - accuracy: 0.9307 - val_loss: 0.0558 - val_accuracy: 0.9828


0.9664399092970521

In [0]:
X_train_gray = color.rgb2gray(X_train).reshape(-1,32,32,1)
X_test_gray = color.rgb2gray(X_test).reshape(-1,32,32,1)

In [0]:
model = get_cnn_v5((32,32,1), num_classes)
model_trained = train_model(model, X_train_gray, y_train)
predict(model_trained, X_test_gray, y_test)

Epoch 1/5
272/272 [==============================] - 13s 46ms/step - loss: 2.5321 - accuracy: 0.2884 - val_loss: 0.6591 - val_accuracy: 0.7757
Epoch 2/5
272/272 [==============================] - 12s 45ms/step - loss: 0.5236 - accuracy: 0.8257 - val_loss: 0.1235 - val_accuracy: 0.9603
Epoch 3/5
272/272 [==============================] - 12s 45ms/step - loss: 0.2353 - accuracy: 0.9238 - val_loss: 0.0657 - val_accuracy: 0.9830
Epoch 4/5
272/272 [==============================] - 12s 45ms/step - loss: 0.1482 - accuracy: 0.9526 - val_loss: 0.0309 - val_accuracy: 0.9910
Epoch 5/5
173/272 [==================>...........] - ETA: 3s - loss: 0.1137 - accuracy: 0.9652